In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=bff3bd29940965961d5e0171470e1c52603ae3ce87f9f279929f0a99d3c37b62
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').appName('spark.com').enableHiveSupport().getOrCreate()

In [ ]:
data = [("Alice",28,"Male"),("Bob",22,"Male"),("Charlie",35,"Male"),("Anna",32,"Female")]
columns = ["name","age","gender"]
df = spark.createDataFrame(data, columns)

In [ ]:
#using dataframeAPI
df.createOrReplaceTempView("people")
df.write.saveAsTable("new_table_name_1")
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)



In [ ]:
#SQL API
spark.sql("CREATE TABLE new_table_name_2 AS SELECT * FROM people")

DataFrame[]

In [ ]:
result = spark.sql("SELECT * FROM people")
result.show()

+-------+---+------+
|   name|age|gender|
+-------+---+------+
|  Alice| 28|  Male|
|    Bob| 22|  Male|
|Charlie| 35|  Male|
|   Anna| 32|Female|
+-------+---+------+



In [ ]:
result = spark.sql("SELECT * FROM people WHERE age > 25")
result.show()

+-------+---+------+
|   name|age|gender|
+-------+---+------+
|  Alice| 28|  Male|
|Charlie| 35|  Male|
|   Anna| 32|Female|
+-------+---+------+



In [ ]:
spark.sql("DESCRIBE new_table_name_2").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|    name|   string|   null|
|     age|   bigint|   null|
|  gender|   string|   null|
+--------+---------+-------+



In [ ]:
#SQL API
spark.sql("SHOW COLUMNS FROM new_table_name_2").show()

+--------+
|col_name|
+--------+
|    name|
|     age|
|  gender|
+--------+



In [ ]:
#Dframe API
from pyspark.sql import functions as F
df.withColumn("new_column",F.lit("some_value")).write.saveAsTable("new_table_name_3")

In [ ]:
#SQL API
spark.sql("ALTER TABLE new_table_name_2 ADD COLUMN new_column STRING")

DataFrame[]

In [ ]:
#Dframe API
new_data = [("David",45,"Male"),("Ella",25,"Female")]
new_df = spark.createDataFrame(new_data, columns)
new_df.write.insertInto("new_table_name_1")

In [ ]:
#SQL API
spark.sql("INSERT INTO TABLE new_table_name_2 VALUES ('David',30,'Male','KEC'),('Ella',25,'Female','KEC')")

DataFrame[]

In [ ]:
#SQL API
spark.sql("SELECT * FROM new_table_name_2").show()

+-------+---+------+----------+
|   name|age|gender|new_column|
+-------+---+------+----------+
|  Alice| 28|  Male|      null|
|    Bob| 22|  Male|      null|
|Charlie| 35|  Male|      null|
|   Anna| 32|Female|      null|
|  David| 30|  Male|       KEC|
|   Ella| 25|Female|       KEC|
+-------+---+------+----------+



In [ ]:
#update query is not possilbe in SQL API
#Dframe API
df = spark.read.table("new_table_name_2")
df.show()

+-------+---+------+----------+
|   name|age|gender|new_column|
+-------+---+------+----------+
|  Alice| 28|  Male|      null|
|    Bob| 22|  Male|      null|
|Charlie| 35|  Male|      null|
|   Anna| 32|Female|      null|
|  David| 30|  Male|       KEC|
|   Ella| 25|Female|       KEC|
+-------+---+------+----------+



In [ ]:
df = df.withColumn("age_plus_5",F.expr("age + 5"))
df.show()

+-------+---+------+----------+----------+
|   name|age|gender|new_column|age_plus_5|
+-------+---+------+----------+----------+
|  Alice| 28|  Male|      null|        33|
|    Bob| 22|  Male|      null|        27|
|Charlie| 35|  Male|      null|        40|
|   Anna| 32|Female|      null|        37|
|  David| 30|  Male|       KEC|        35|
|   Ella| 25|Female|       KEC|        30|
+-------+---+------+----------+----------+



In [ ]:
updated_df = df.withColumn("age",F.expr("age + 1"))
updated_df.show()

+-------+---+------+----------+----------+
|   name|age|gender|new_column|age_plus_5|
+-------+---+------+----------+----------+
|  Alice| 29|  Male|      null|        33|
|    Bob| 23|  Male|      null|        27|
|Charlie| 36|  Male|      null|        40|
|   Anna| 33|Female|      null|        37|
|  David| 31|  Male|       KEC|        35|
|   Ella| 26|Female|       KEC|        30|
+-------+---+------+----------+----------+



In [ ]:
updated_df = df.withColumn("is_adult",F.when(F.expr("age>=18"),"Yes").otherwise("No"))
updated_df.show()

+-------+---+------+----------+----------+--------+
|   name|age|gender|new_column|age_plus_5|is_adult|
+-------+---+------+----------+----------+--------+
|  Alice| 28|  Male|      null|        33|     Yes|
|    Bob| 22|  Male|      null|        27|     Yes|
|Charlie| 35|  Male|      null|        40|     Yes|
|   Anna| 32|Female|      null|        37|     Yes|
|  David| 30|  Male|       KEC|        35|     Yes|
|   Ella| 25|Female|       KEC|        30|     Yes|
+-------+---+------+----------+----------+--------+



In [ ]:
update_cnt = (F.col("name")=="Bob")
updated_df = df.withColumn("age",F.when(update_cnt,25).otherwise(F.col("age")))
updated_df.show()

+-------+---+------+----------+----------+
|   name|age|gender|new_column|age_plus_5|
+-------+---+------+----------+----------+
|  Alice| 28|  Male|      null|        33|
|    Bob| 25|  Male|      null|        27|
|Charlie| 35|  Male|      null|        40|
|   Anna| 32|Female|      null|        37|
|  David| 30|  Male|       KEC|        35|
|   Ella| 25|Female|       KEC|        30|
+-------+---+------+----------+----------+



In [ ]:
job_value = "Engineer"
df_with_job = df.withColumn("job",F.lit(job_value))
df_with_job.show()

+-------+---+------+----------+----------+--------+
|   name|age|gender|new_column|age_plus_5|     job|
+-------+---+------+----------+----------+--------+
|  Alice| 28|  Male|      null|        33|Engineer|
|    Bob| 22|  Male|      null|        27|Engineer|
|Charlie| 35|  Male|      null|        40|Engineer|
|   Anna| 32|Female|      null|        37|Engineer|
|  David| 30|  Male|       KEC|        35|Engineer|
|   Ella| 25|Female|       KEC|        30|Engineer|
+-------+---+------+----------+----------+--------+



In [ ]:
update_cnt = (F.col("name").isin(["Alice","Charlie"]))
age_update_expr = F.when(update_cnt,F.col("age")+5).otherwise(F.col("age"))
updated_df = df_with_job.withColumn("job",job_update_expr).withColumn("age",age_update_expr)

In [ ]:
updated_df.show()

+-------+---+------+----------+----------+---+
|   name|age|gender|new_column|age_plus_5|job|
+-------+---+------+----------+----------+---+
|  Alice| 33|  Male|      null|        33| 33|
|    Bob| 22|  Male|      null|        27| 22|
|Charlie| 40|  Male|      null|        40| 40|
|   Anna| 32|Female|      null|        37| 32|
|  David| 30|  Male|       KEC|        35| 30|
|   Ella| 25|Female|       KEC|        30| 25|
+-------+---+------+----------+----------+---+



In [ ]:
#Delete is not working
update_df = updated_df.filter(df["name"]!="Bob")
update_df.show()

+-------+---+------+----------+----------+---+
|   name|age|gender|new_column|age_plus_5|job|
+-------+---+------+----------+----------+---+
|  Alice| 33|  Male|      null|        33| 33|
|Charlie| 40|  Male|      null|        40| 40|
|   Anna| 32|Female|      null|        37| 32|
|  David| 30|  Male|       KEC|        35| 30|
|   Ella| 25|Female|       KEC|        30| 25|
+-------+---+------+----------+----------+---+

